# Automated ML Embedding
In this notebook, we'll use Azure ML to train a model using our graph embedding as an additional feature.

In [1]:
%pip install graphdatascience
%pip install azure-ai-ml
%pip install azureml-mlflow
%pip install mlflow
%pip install mltable

Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 4.0 MB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 5.6 MB/s  eta 0:00:01
     |████████████████████████████████| 383 kB 57.8 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 2.3 MB/s  eta 0:00:01
     |████████████████████████████████| 76 kB 5.5 MB/s  eta 0:00:01
     |████████████████████████████████| 252 kB 58.7 MB/s eta 0:00:01
     |████████████████████████████████| 137 kB 55.7 MB/s eta 0:00:01
     |████████████████████████████████| 238 kB 53.6 MB/s eta 0:00:01
  Created wheel for strictyaml: filename=strictyaml-1.6.1-py3-none-any.whl size=123915 sha256=68876a514672426a2e98e212dfab761a393dd4a3b207b10dbdc1f6fd3c5a29d2
  Stored in directory: /home/azureuser/.cache/pip/wheels/24/8b/79/d11bb76d63bc1b37df7bd35dc1e47577eb926b7eb1b619fc2d
Successfully built strictyaml
ERROR: fbprophet 0.7.1 requires cmdstanpy==0.9.5, which is not installed.
ERROR:

## Azure ML Connection
Let's setup our Azure ML connection.

### Import the required libraries

In [2]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.identity import AzureCliCredential
from azure.ai.ml import automl, Input, MLClient

from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.automl import (
    classification,
    ClassificationPrimaryMetrics,
    ClassificationModels,
)

Class SystemCreatedStorageAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SystemCreatedAcrAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Workspace details

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

By default, we try to use the by default workspace configuration (available out-of-the-box in Compute Instances) or from any Config.json file you might have copied into the folders structure.
If no Config.json is found, then you need to manually introduce the subscription_id, resource_group and workspace when creating MLClient .

In [3]:
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "4109ec49-0ff1-497c-b615-2de74128cf76"
    resource_group = "ev-az-ws"
    workspace = "azure-ml-ws"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


We could not find config.json in: . or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.


In [4]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

output = {}
output["Workspace"] = ml_client.workspace_name
output["Subscription ID"] = ml_client.connections._subscription_id
output["Resource Group"] = workspace.resource_group
output["Location"] = workspace.location
output

{'Workspace': 'azure-ml-ws',
 'Subscription ID': '4109ec49-0ff1-497c-b615-2de74128cf76',
 'Resource Group': 'ev-az-ws',
 'Location': 'southeastasia'}

## Upload Training Data MLTable
Now we're going to upload the training data Azureml

In [5]:
# MLTable definition file
with open('./data/training-mltable-folder/MLTable', 'w') as f:
    f.write("""
        paths:
            - file: ./train.csv
        transformations:
            - read_delimited:
                    delimiter: ','
                    encoding: 'ascii'
    """)

    # MLTable definition file
with open('./data/test-mltable-folder/MLTable', 'w') as f:
    f.write("""
        paths:
            - file: ./test.csv
        transformations:
            - read_delimited:
                    delimiter: ','
                    encoding: 'ascii'
    """)
    
# MLTable definition file
with open('./data/validation-mltable-folder/MLTable', 'w') as f:
    f.write("""
        paths:
            - file: ./validate.csv
        transformations:
            - read_delimited:
                    delimiter: ','
                    encoding: 'ascii'
    """)

In [6]:
# Create MLTables for training dataset

my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./data/training-mltable-folder"
)

In [7]:
# General job parameters
compute_name = "azure-ml-compute"
max_trials = 5
exp_name = "asset-prediction-experiment"

## Setting up the Classification Job
After uploading the dataset, you can invoke AutoML to find the best ML pipeline to train a model on this dataset.

In [8]:
# Create the AutoML classification job with the related factory-function.

classification_job = automl.classification(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name="target",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True,
    tags={"classification_task": "sec"},
)

# Limits are all optional
classification_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=max_trials,
    # max_concurrent_trials = 4,
    # max_cores_per_trial: -1,
    enable_early_termination=True,
)

# Training properties are optional
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"],
    enable_onnx_compatible_models=True,
)

In [9]:
# Submit the AutoML job (CDLTLL: Is it ml_client.create_or_update(classification_job))
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

# Get a URL for the status of the job
# returned_job.services["Studio"].endpoint

Uploading training-mltable-folder (60.77 MBs): 100%|██████████| 60769076/60769076 [00:06<00:00, 9135971.28it/s]




Created job: ClassificationJob({'log_verbosity': <LogVerbosity.INFO: 'Info'>, 'target_column_name': 'target', 'weight_column_name': None, 'validation_data_size': None, 'cv_split_column_names': None, 'n_cross_validations': 5, 'test_data_size': None, 'task_type': <TaskType.CLASSIFICATION: 'Classification'>, 'training_data': {'type': 'mltable', 'path': 'azureml://datastores/workspaceblobstore/paths/LocalUpload/e0efeda771cb324242e1f787021ede4c/training-mltable-folder'}, 'validation_data': {'type': 'mltable'}, 'test_data': None, 'environment_id': None, 'environment_variables': None, 'outputs': {}, 'type': 'automl', 'status': 'NotStarted', 'log_files': None, 'name': 'kind_toe_xxlgp6yd6s', 'description': None, 'tags': {'classification_task': 'sec'}, 'properties': {}, 'id': '/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/resourceGroups/ev-az-ws/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-ws/jobs/kind_toe_xxlgp6yd6s', 'Resource__source_path': None, 'base_path': '/mnt/ba

## Launching the AutoML Job

This job is going to take close to 30 minutes. At this point, you might want to set the rest of this notebook to run and move on to "Lab 7 - Cleanup."  At this point you can delete your Neo4j deployment without impacting the running AutoML job.

In [10]:
ml_client.jobs.stream(returned_job.name)

RunId: kind_toe_xxlgp6yd6s
Web View: https://ml.azure.com/runs/kind_toe_xxlgp6yd6s?wsid=/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/resourcegroups/ev-az-ws/workspaces/azure-ml-ws

Execution Summary
RunId: kind_toe_xxlgp6yd6s
Web View: https://ml.azure.com/runs/kind_toe_xxlgp6yd6s?wsid=/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/resourcegroups/ev-az-ws/workspaces/azure-ml-ws



In [11]:
# Get a URL for the status of the job
returned_job.services["Studio"].endpoint

'https://ml.azure.com/runs/kind_toe_xxlgp6yd6s?wsid=/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/resourcegroups/ev-az-ws/workspaces/azure-ml-ws&tid=54e85725-ed2a-49a4-a19e-11c8d29f9a0f'

In [12]:
print(returned_job.name)

kind_toe_xxlgp6yd6s


# It's optional to proceed to the sections below

## Retrieve the Best Trial (Best Model's trial/run)
Use the MLFLowClient to access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Trial.

## Initialize MLFlow Client
The models and artifacts that are produced by AutoML can be accessed via the MLFlow interface. 
Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

*IMPORTANT*, you need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow

    pip install mlflow

### Obtain the tracking URI for MLFlow

In [13]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://southeastasia.api.azureml.ms/mlflow/v1.0/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/resourceGroups/ev-az-ws/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-ws


In [14]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))


Current tracking uri: azureml://southeastasia.api.azureml.ms/mlflow/v1.0/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/resourceGroups/ev-az-ws/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-ws


In [15]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

### Get the AutoML parent Job

In [16]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "b4e95546-0aa1-448e-9ad6-002e3207b4fc"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.7207708808230493,
 'AUC_micro': 0.7481093473680807,
 'AUC_weighted': 0.7207708808230491,
 'accuracy': 0.6805897999454622,
 'average_precision_score_macro': 0.7197956082144596,
 'average_precision_score_micro': 0.7497792135317148,
 'average_precision_score_weighted': 0.7366045864206219,
 'balanced_accuracy': 0.6218805508469197,
 'f1_score_macro': 0.615276180952615,
 'f1_score_micro': 0.6805897999454622,
 'f1_score_weighted': 0.6481623507965925,
 'log_loss': 0.5860737950122005,
 'matthews_correlation': 0.30290655261521593,
 'norm_macro_recall': 0.2437611016938393,
 'precision_score_macro': 0.6882082509123618,
 'precision_score_micro': 0.6805897999454622,
 'precision_score_weighted': 0.6856571778896907,
 'recall_score_macro': 0.6218805508469197,
 'recall_score_micro': 0.6805897999454622,
 'recall_score_weighted': 0.6805897999454622,
 'weighted_accuracy': 0.7344501599647202}, params={}, tags={'automl_best_child_run_id': 'kind_toe_xx

In [17]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

{'classification_task': 'sec', 'model_explain_run': 'best_run', 'pipeline_id': '', 'score': '', 'predicted_cost': '', 'fit_time': '', 'training_percent': '', 'iteration': '', 'run_preprocessor': '', 'run_algorithm': '', 'automl_best_child_run_id': 'kind_toe_xxlgp6yd6s_4', 'model_explain_best_run_child_id': 'kind_toe_xxlgp6yd6s_4', 'mlflow.rootRunId': 'kind_toe_xxlgp6yd6s', 'mlflow.runName': 'kind_toe_xxlgp6yd6s', 'mlflow.user': 'Ezhil Vezhavendhan'}


## Get the AutoML best child run

In [18]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

Found best child run id:  kind_toe_xxlgp6yd6s_4
Best child run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.7207708808230493,
 'AUC_micro': 0.7481093473680807,
 'AUC_weighted': 0.7207708808230491,
 'accuracy': 0.6805897999454622,
 'average_precision_score_macro': 0.7197956082144596,
 'average_precision_score_micro': 0.7497792135317148,
 'average_precision_score_weighted': 0.7366045864206219,
 'balanced_accuracy': 0.6218805508469197,
 'f1_score_macro': 0.615276180952615,
 'f1_score_micro': 0.6805897999454622,
 'f1_score_weighted': 0.6481623507965925,
 'log_loss': 0.5860737950122005,
 'matthews_correlation': 0.30290655261521593,
 'norm_macro_recall': 0.2437611016938393,
 'precision_score_macro': 0.6882082509123618,
 'precision_score_micro': 0.6805897999454622,
 'precision_score_weighted': 0.6856571778896907,
 'recall_score_macro': 0.6218805508469197,
 'recall_score_micro': 0.6805897999454622,
 'recall_score_weighted': 0.6805897999454622,
 'weighted_accuracy': 0.7344501599647202}, param

## Get best model run's metrics

Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [19]:
best_run.data.metrics

{'f1_score_micro': 0.6805897999454622,
 'balanced_accuracy': 0.6218805508469197,
 'AUC_weighted': 0.7207708808230491,
 'AUC_micro': 0.7481093473680807,
 'precision_score_macro': 0.6882082509123618,
 'recall_score_micro': 0.6805897999454622,
 'norm_macro_recall': 0.2437611016938393,
 'recall_score_weighted': 0.6805897999454622,
 'precision_score_micro': 0.6805897999454622,
 'f1_score_macro': 0.615276180952615,
 'accuracy': 0.6805897999454622,
 'f1_score_weighted': 0.6481623507965925,
 'matthews_correlation': 0.30290655261521593,
 'precision_score_weighted': 0.6856571778896907,
 'average_precision_score_macro': 0.7197956082144596,
 'average_precision_score_micro': 0.7497792135317148,
 'weighted_accuracy': 0.7344501599647202,
 'average_precision_score_weighted': 0.7366045864206219,
 'log_loss': 0.5860737950122005,
 'recall_score_macro': 0.6218805508469197,
 'AUC_macro': 0.7207708808230493}

## Download the best model locally

Access the results (such as Models, Artifacts, Metrics) of a previously completed AutoML Run.

In [20]:
import os

# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [21]:
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print("Artifacts downloaded in: {}".format(local_path))
print("Artifacts: {}".format(os.listdir(local_path)))

Artifacts downloaded in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/azure-ml-compute/code/Users/ezhil.vezhavendhan/notebooks/Lab 6 - Graph Data Science/artifact_downloads/outputs
Artifacts: ['conda_env_v_1_0_0.yml', 'engineered_feature_names.json', 'env_dependencies.json', 'featurization_summary.json', 'generated_code', 'internal_cross_validated_models.pkl', 'mlflow-model', 'model.pkl', 'pipeline_graph.json', 'run_id.txt', 'scoring_file_pbi_v_1_0_0.py', 'scoring_file_v_1_0_0.py', 'scoring_file_v_2_0_0.py']


In [22]:
# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

['conda.yaml', 'MLmodel', 'model.pkl', 'python_env.yaml', 'requirements.txt']

## Register Best Model and Deploy
### Create managed online endpoint

In [23]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)
from azure.ai.ml.constants import ModelType

In [24]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "asset-prediction-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Asset Prediction endpoint for mlflow model",
    auth_mode="key",
    tags={"lab": "neo4j"},
)

In [25]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://asset-prediction-10210502764044.southeastasia.inference.ml.azure.com/score', 'openapi_uri': 'https://asset-prediction-10210502764044.southeastasia.inference.ml.azure.com/swagger.json', 'name': 'asset-prediction-10210502764044', 'description': 'Asset Prediction endpoint for mlflow model', 'tags': {'lab': 'neo4j', 'Owner Department': 'Business Development'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/resourcegroups/ev-az-ws/providers/microsoft.machinelearningservices/workspaces/azure-ml-ws/onlineendpoints/asset-prediction-10210502764044', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/providers/Microsoft.MachineLearningServices/locations/southeastasia/mfeOperationsStatus/oe:bdd85ba3-69b9-4e0b-b707-4a0160c03dbb:c855adfe-f86c-416a-a2e2-9be8b127097f?api-version=20

### Register best model and deploy

In [26]:
model_name = "asset-prediction-model"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/model.pkl",
    name=model_name,
    description="GDS mlflow model",
)

# for downloaded file
# model = Model(path="artifact_downloads/outputs/model.pkl", name=model_name)

registered_model = ml_client.models.create_or_update(model)

In [27]:
registered_model.id

'/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/resourceGroups/ev-az-ws/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-ws/models/asset-prediction-model/versions/1'

#### Configure environment and code configuration

In [28]:
env = Environment(
    name="automl-tabular-env",
    description="environment for automl inference",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1",
    conda_file="artifact_downloads/outputs/conda_env_v_1_0_0.yml",
)

In [29]:
code_configuration = CodeConfiguration(
    code="artifact_downloads/outputs/", scoring_script="scoring_file_v_2_0_0.py"
)

#### Deploy

In [30]:
deployment = ManagedOnlineDeployment(
    name="asset-prediction-deploy",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    environment=env,
    code_configuration=code_configuration,
    instance_type="Standard_DS2_V2",
    instance_count=1,
)

In [31]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

Check: endpoint asset-prediction-10210502764044 exists
Uploading outputs (35.32 MBs): 100%|██████████| 35321588/35321588 [00:00<00:00, 57709258.77it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


................................................................................................................................................................................................................................................................................

ManagedOnlineDeployment({'private_network_connection': False, 'data_collector': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'asset-prediction-10210502764044', 'type': 'Managed', 'name': 'asset-prediction-deploy', 'description': None, 'tags': {'Owner Department': 'Business Development'}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/providers/Microsoft.MachineLearningServices/locations/southeastasia/mfeOperationsStatus/od:bdd85ba3-69b9-4e0b-b707-4a0160c03dbb:698cdd9c-8ac1-41b3-9eef-96d91e91f8f1?api-version=2022-02-01-preview'}, 'id': '/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/resourceGroups/ev-az-ws/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-ws/onlineEndpoints/asset-prediction-10210502764044/deployments/asset-prediction-deploy', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/azure-ml-compute/code/Users/ezhil.vezhavendhan/n

In [32]:
# asset-prediction deployment to take 100% traffic
endpoint.traffic = {"asset-prediction-deploy": 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://asset-prediction-10210502764044.southeastasia.inference.ml.azure.com/score', 'openapi_uri': 'https://asset-prediction-10210502764044.southeastasia.inference.ml.azure.com/swagger.json', 'name': 'asset-prediction-10210502764044', 'description': 'Asset Prediction endpoint for mlflow model', 'tags': {'lab': 'neo4j', 'Owner Department': 'Business Development'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/resourcegroups/ev-az-ws/providers/microsoft.machinelearningservices/workspaces/azure-ml-ws/onlineendpoints/asset-prediction-10210502764044', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/4109ec49-0ff1-497c-b615-2de74128cf76/providers/Microsoft.MachineLearningServices/locations/southeastasia/mfeOperationsStatus/oe:bdd85ba3-69b9-4e0b-b707-4a0160c03dbb:e09f4c48-3530-44d6-98e8-4995789bee9a?api-version=20

### Test the deployment

In [39]:
# test the blue deployment with some sample data
import pandas as pd

test_data = pd.read_csv("./data/test-mltable-folder/test.csv").head(20)

test_data_json = test_data.to_json(orient="records", indent=4)
data = (
    '{ \
          "Inputs": {"data": '
    + test_data_json
    + '}, \
          "GlobalParameters": { "method": "predict" } \
        }'
)

request_file_name = "test-request.json"

with open(request_file_name, "w") as request_file:
    request_file.write(data)

ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="asset-prediction-deploy",
    request_file=request_file_name,
)

'{"Results": [true, false, false, false, false, false, false, false, false, false, false, false, true, false, false, false, false, false, false, false]}'

### Get endpoint details

In [40]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'asset-prediction-deploy': 100}
https://asset-prediction-10210502764044.southeastasia.inference.ml.azure.com/score


In [ ]:
### Delete the deploy
ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()

............................................